In [1]:
import pandas as pd
import glob

In [6]:
all_results = glob.glob("Member of the City Council*.tsv")
all_supplements = glob.glob("Supplement*.tsv")

In [65]:
dfs = []

for f1, f2 in zip(sorted(all_results), sorted(all_supplements)):
  df = pd.read_csv(f1, sep='\t')
  df2 = pd.read_csv(f2, sep='\t')
  #district	reg_dems	precincts	reporting	votes	total	candidate
  
  precincts = len(df)
  tmp = ((df.reporting.str.strip('%')).astype(float)/100)
  reporting = len(tmp[tmp > 0])
  total = int(df.total.sum())
  district = f2.split(" - ")[1].split(".")[0]
  reg_dems = df2.total_reg_dems.sum()
  
  candidate_cols = df.columns[df.columns.str.contains("candidate\|")]
  candidate_order = [c.replace("candidate|", "") for c in candidate_cols]
  
  summary = pd.melt(df, value_vars=candidate_cols, value_name="votes", var_name="candidate").groupby("candidate").sum().reset_index()
  summary.candidate = [c.replace("candidate|", "") for c in summary.candidate]
  summary["district"] = district
  summary["reg_dems"] = reg_dems
  summary["precincts"] = precincts
  summary["reporting"] = reporting
  summary["total"] = total
  summary.votes = summary.votes.astype(int)
  
  
  sorterIndex = dict(zip(candidate_order, range(len(candidate_order))))
  summary['rank'] = summary['candidate'].map(sorterIndex)
  summary.sort_values("rank", inplace=True)
  summary.drop("rank", axis=1, inplace=True)
  dfs.append(summary)
  
final = pd.concat(dfs, ignore_index=True)

In [69]:
final.to_csv("2021/summary.tsv", sep='\t', index=False)